In [8]:
import os
import argparse
import io
import glob
import hashlib
from pathlib import Path
from PIL import Image
import subprocess
import cv2

import tensorflow as tf
from object_detection.utils import dataset_util


In [9]:
DATASET_PATH = "/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset"
TF_FILE_PATH = "/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/tf_data_new.tfrecord"

In [10]:
class_names = {0:"door",
              1:"handle",
              2:"cabinet door",
              3:"refrigerator door"}

In [21]:
def create_tf_example(img_path:str,
                      annotation_path:str,
                     class_names:dict):
    
    # ------------------------------------------------------------
    # Read the image
    with tf.gfile.GFile(img_path, 'rb') as fid:
        encoded_img = fid.read()

    encoded_img_io = io.BytesIO(encoded_img)
    image = Image.open(encoded_img_io)
    img_width, img_height = image.size
    
    key = hashlib.sha256(encoded_img).hexdigest()
    
    fn = Path(img_path).stem
    
    # ------------------------------------------------------------
    # Read the annotation
    x_1,y_1,x_2,y_2 = [],[],[],[]
    clss_list = []
    with open(annotation_path, 'r') as li:
        annotations = li.readlines()
        for i in annotations:
#             print(list(map(float,i.strip().split(' '))))
            label,x,y,w,h = list(map(float,i.strip().split(' ')))
            x_1.append(float(x - (w / 2)))
            y_1.append(float(y - (h / 2)))
            x_2.append(float(x + (w / 2)))
            y_2.append(float(y + (h / 2)))
            clss_list.append(int(label))
    
    # --------------------------------------------------------------
    # TODO(user): Populate the following variables from your example.
    width,height  = image.size    # Image width , Image height
    filename = fn.encode('utf8') # Filename of the image. Empty if image is not from file
    encoded_image_data = encoded_img # Encoded image bytes
    image_format = b'jpeg' # b'jpeg' or b'png'

    xmins = x_1 # List of normalized left x coordinates in bounding box (1 per box)
    ymins = y_1 # List of normalized top y coordinates in bounding box (1 per box)
    xmaxs = x_2 # List of normalized right x coordinates in bounding box (1 per box)
    ymaxs = y_2 # List of normalized bottom y coordinates in bounding box (1 per box)

    classes_text = [k.encode('utf8') for k in class_names.values()] # List of string class name of bounding box (1 per box)
    classes = clss_list # List of integer class id of bounding box (1 per box)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
      }))
    
    return tf_example

In [22]:
writer =  tf.io.TFRecordWriter(TF_FILE_PATH)

img_file_list = glob.glob(os.path.join(DATASET_PATH,"*.jpg"), recursive=True)

for i,img in enumerate(img_file_list) :
    name = Path(img).stem

    img_path = os.path.join(DATASET_PATH , name + ".jpg")
    annotation_path = os.path.join(DATASET_PATH ,name  + ".txt")
    
    print(img_path)
    
    tf_record = create_tf_example(img_path,annotation_path,class_names)
    writer.write(tf_record.SerializeToString())
    
    print(i," ",name," written to tf record.")

/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/0e3381d79d5b1ea1.jpg
(1024, 687)
[2.0, 0.25, 0.366667, 0.06875, 0.269444]
[2.0, 0.313672, 0.368056, 0.058594, 0.261111]
[2.0, 0.256641, 0.5625, 0.061719, 0.1]
[2.0, 0.316797, 0.553472, 0.064844, 0.093056]
[2.0, 0.289062, 0.896528, 0.034375, 0.206944]
[2.0, 0.340234, 0.910417, 0.042969, 0.176389]
[2.0, 0.389453, 0.940278, 0.041406, 0.119444]
[2.0, 0.445312, 0.952083, 0.0625, 0.095833]
[2.0, 0.464063, 0.365972, 0.035937, 0.168056]
[2.0, 0.509375, 0.368056, 0.042188, 0.155556]
[2.0, 0.546484, 0.370139, 0.044531, 0.151389]
[2.0, 0.585156, 0.376389, 0.042188, 0.155556]
[2.0, 0.168359, 0.646528, 0.082031, 0.031944]
[2.0, 0.175, 0.695833, 0.08125, 0.044444]
[2.0, 0.171875, 0.752083, 0.090625, 0.040278]
[2.0, 0.173047, 0.360417, 0.052344, 0.201389]
[2.0, 0.114453, 0.351389, 0.050781, 0.205556]
[2.0, 0.050391, 0.333333, 0.057031, 0.233333]
[1.0, 0.01875, 0.415972, 0.0125, 0.068056]
[2.0, 0.613672, 0.60625, 0.027344, 0.059722]
[2.0, 0.571094, 